# Auto-Transformer

## Input

### User Input

- Input Voltage $V_{in}=115 $ $V$
- Output Power $P_o=250$ $[Watts]$
- Output Voltage, $V_o = 115$ volts
- Frequency $f=47$ $Hz$
- Efficiency $\eta=95\%$
- Operating flux density $B_{ac}=1.6$
- Temperature rise goal, $T_r=30^oC$
- Window utilization, $K_u=0.4$
- Core Material = $Silicon M6X$
- Lamination Number = $EI-150$
- Current Density $J=250$ $ [amps/cm^2]$
- Bobbin thickness = $1.5$ $mm$
- Regulation $\alpha = 5$ $\%$

### Constants 

- Window utilization $K_u = 0.4$

In [1]:
Frequency = 47 # Hz
Temperature_rise_goal = 30 # degree Celcius
Output_power = 250 # watts
Ouptut_voltage = 115 # volts
Efficiency = 95 # %
Input_voltage = 115 # volts
Bobbin_thickness = 1.5 
Regulation = 5 # percent
K_f = 4.44
K_u = 0.4
B_ac = 1.6
J = 250 # amps / cm^2


## Apparent Power $P_t$

Calculating the transformer apparent power, $P_t$

$$ P_t=P_o(\dfrac{1}{\eta} + 1), [watts]$$

$$P_t=250[(1/0.95)+1]$$

$$P_t=513, [watts]$$

In [2]:
Apparent_power = Output_power * (1/(0.01*Efficiency) + 1)
print('Apparent power: ' + str(Apparent_power) + ' Watts')

Apparent power: 513.1578947368421 Watts


## Calculation of Area Product

$$J=\dfrac{P_t (10^4)}{K_f K_u B_{ac} f A_p} $$

$$A_p=\dfrac{P_t (10^4)}{K_f K_u B_{ac} f J} $$

For initial case :

$$K_f = 4.44$$
$$K_u = 0.4 $$
$$B_{ac} = 1.6$$
$$f= 47$$
$$J=250$$

In [3]:
Area_product = (Apparent_power*(10**4))/(K_f * K_u * B_ac * J * Frequency)
print('Theoretical Area product: ' + str(Area_product) + ' cm^4') # cm^4

Theoretical Area product: 153.6916255561272 cm^4


In [4]:
import pandas as pd
import numpy as np 

In [5]:
lamination_data = pd.read_csv('./EI-Laminations.csv')
lamination_data.head()

,Type,A,B,C,D,E,Tongue,Winding-width,Winding-length,K-ratio,Remarks,Fig-No
0,SB1R,8.00,28.00,25.00,4.00,4.00,8.00,6.000,17.00,816.000000,-,1
1,SB3R,9.50,35.00,29.50,5.00,5.00,9.50,7.750,19.50,1435.687500,-,1
2,17,12.70,38.10,31.75,6.35,6.35,12.70,6.350,19.05,1536.287250,-,1
3,74,18.07,53.93,45.11,8.94,8.94,18.07,8.990,27.23,4423.494439,WW 8.9,2
4,12A,15.87,47.62,38.68,7.93,7.93,15.87,7.945,22.82,2877.308763,-,1


- A - in $mm$
- B - in $mm$
- C - in $mm$
- D - in $mm$
- Tongue - in $mm$
- Winding-width - in $mm$
- Winding-length - in $mm$
- K - in $mm^3$

## Input current  Bare wire area Diameter of insulated wire

In [6]:
# Input_current = Apparent_power / (Input_voltage*Efficiency)
Input_current = Apparent_power / (Input_voltage * 0.01* Efficiency)

# Primary bare wire area A_wp = Input_current / Current_density (J)
A_wp = Input_current / J

# bare area in sqcm so convert it into sqmm
A_wp_in_sqmm = A_wp * 100

# select the swg the data 
swg_data = pd.read_csv('EMD - Sheet1.csv')

# select the swg nearest to the required conductor area
required_swg_primary = swg_data.iloc[(swg_data['Normal Conductor Area mm²'] - A_wp_in_sqmm).abs().argsort()[:1]]

# Diameter of primary wire with insulation enamel
diameter_of_primary_wire_with_insulation = required_swg_primary['Medium Covering Max']

# Print the results 
print('Input current ' + str(Input_current) + ' [amps]')
print('Priamry Bare Area: ' + str(A_wp*100)+ ' mm²') 
print('SWG: ' + str(required_swg_primary['SWG'].to_string(index=False)))
print('Primary wire Diameter with enamel: ' + str(diameter_of_primary_wire_with_insulation.max()) + ' mm')



Input current 4.69709743466217 [amps]
Priamry Bare Area: 1.8788389738648679 mm²
SWG: 16.0
Primary wire Diameter with enamel: 1.737 mm


## Select lamination and vary the area product

In [12]:
stack_data = []
# Awg = []

for lamination in lamination_data['Type']:
    selected_lamination = lamination_data[lamination_data['Type'] == lamination]
    for x in range(60, 141, 5):
        
        # present area product with some part of Area product
        present_Area_product =  x * 0.01 * Area_product 
        
        # calculate stack using the area product and K-ratio 
        # present_Area_product in cm^4 
        # K-ratio in mm^3
        stack = present_Area_product * 1000 / selected_lamination['K-ratio'].max() 
        # stack in cm
        

        # Now convert stack, tongue, winding length, winding width to mm
        stack = stack * 10
        tongue = selected_lamination['Tongue'].max()
        winding_length = selected_lamination['Winding-length'].max()
        winding_width = selected_lamination['Winding-width'].max()
        
        if stack < 5 * selected_lamination['Tongue'].max():
            before_round_stack = stack 

            if stack < 5:
                stack = 5.0
            elif stack%5 == 0:
                stack = stack
            elif stack%5 <= 2.5:
                stack = stack - stack % 5
            elif stack%5 > 2.5:
                stack = stack - stack % 5 + 5
            # print('Before rounding stack: ' + str(before_round_stack) + ' | After rounding stack: ' + str(stack) + ' | Lamination: ' + str(selected_lamination['Type'].max()))

            # Core Area A_c = tongue[mm] * stack[mm] * 0.01 in cm^2 
            A_c = stack * selected_lamination['Tongue'].max() * 0.01  # in cm^2
            # print('Core Area in cm^2: ' + str(A_c) + 'cm^2 | Lamination : ' + str(selected_lamination['Type'].max()) + '\n -------------------------------------------------------------')

            # Number of primary of turns = input voltage / (k_f * B_ac * Frequency * A_c)
            Number_of_primary_turns = (Input_voltage* 10**4) / (K_f * B_ac * Frequency * A_c)
            # print('Core Area in cm^2: ' + str(A_c) + 'cm^2 | Lamination : ' + str(selected_lamination['Type'].max()) + ' | Primary turns : ' + str(Number_of_primary_turns))

            table_data_stack_and_tongue = {
                'Area Product ': present_Area_product,
                'Stack mm': stack,
                'Tongue mm': selected_lamination['Tongue'].max(),
                'Lamination': selected_lamination['Type'].max(),
                'N_p': Number_of_primary_turns,
                'Tongue * Stack mm²':  selected_lamination['Tongue'].max() * stack,
                'Core Area A_c cm²': A_c 
            }
            stack_data.append(table_data_stack_and_tongue)


df = pd.DataFrame(stack_data)
df      
                        

,Area Product,Stack mm,Tongue mm,Lamination,N_p,Tongue * Stack mm²,Core Area A_c cm²
0,92.214975,85.0,22.22,31,182.361875,1888.7,18.887
1,99.899557,95.0,22.22,31,163.165888,2110.9,21.109
2,107.584138,100.0,22.22,31,155.007594,2222.0,22.220
3,115.268719,110.0,22.22,31,140.915994,2444.2,24.442
4,92.214975,75.0,25.40,15,180.801508,1905.0,19.050
...,...,...,...,...,...,...,...
297,184.429951,5.0,44.00,S222,1565.576699,220.0,2.200
298,192.114532,5.0,44.00,S222,1565.576699,220.0,2.200
299,199.799113,10.0,44.00,S222,782.788349,440.0,4.400
300,207.483695,10.0,44.00,S222,782.788349,440.0,4.400


## For cost calculation
